# Sales Metrics and Data Insights from Brazil’s E-Commerce Market

**Author:** Arturo Alejandro Díaz Barbosa  
**Date:** June 2025  

**Data Source:**  
Brazilian E-Commerce Public Dataset by Olist  
(https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce)

---

## Dataset Description

This project uses the Olist Brazilian e-commerce dataset, which contains over 100,000 orders from September 2016 to August 2018 across multiple marketplaces in Brazil. It includes detailed tables on customers, orders, order items, payments, reviews, products, sellers, geolocation, and category translations. The rich schema allows analysis of sales performance, customer behavior, delivery efficiency, and product reviews from multiple angles.

---

## Objective

This analysis aims to explore and visualize key aspects of Olist’s sales data, answering questions such as:  
1. Which product categories generate the highest revenue and how do they trend over time?  
2. What are the daily and monthly sales patterns, and when are the peak order periods?  
3. How do sales and order volumes distribute across Brazilian states and cities?  
4. What is the average delivery performance by region for both customers and sellers?  
5. How do customer cohorts behave over time in terms of repeat purchases and revenue?

---

## Key Findings

- **Strong Revenue Growth:**  
  – 99,440 orders generated 1.26 M between Sep 2016 and Aug 2018, with revenues rising from 49.8K (2016) to 6.17 M (2017) and \$7.39 M (2018).

- **Seasonal Peaks:**  
  – May saw a notable jump in Health & Beauty sales (Mother’s Day), and November spiked in Watches & Gifts (Black Friday, peaking on the 24th).

- **Top Categories:**  
  – **Health & Beauty** led total revenue (~\$1.26 M).  
  – **Bed & Bath Table** topped order volume (11 115 orders, 15.5%).

- **Regional Concentration:**  
  – São Paulo accounted for 41 K orders (\$5.20 M); Rio de Janeiro and Minas Gerais followed.  
  – Roraima and other remote states remain under-penetrated (< 8 K sales).

- **Delivery Gaps:**  
  – Avg. delivery: 18.8 days to customers vs. 15.2 days seller dispatch.  
  – São Paulo fastest (8.8/12.3 days); Bahia lagged by 5 days.

- **Low Repeat Purchase:**  
  – 96.9 % of customers bought once; only 2.9 % returned for a second order.


## Import required libraries

In [1]:
import pandas as pd 
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import re
from sqlalchemy import text
import inspect

## Database Connection Setup

- Load database credentials securely using environment variables with `python-dotenv`.
- Construct a PostgreSQL connection string using `SQLAlchemy`.
- Initialize a connection engine for executing SQL queries from Python.
- This approach keeps sensitive information (e.g., user, password) out of the codebase.


In [2]:
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

db_connection = f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}'
engine = create_engine(db_connection)

## Dataset File Mapping

- Define a list of all raw CSV files provided in the Olist e-commerce dataset.
- Assign clear, readable aliases for each file to improve code clarity.
- These aliases will be used as table names when loading the data into SQL or working with DataFrames.


In [3]:
files = ['olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_order_items_dataset.csv',\
         'olist_order_payments_dataset.csv','olist_order_reviews_dataset.csv', 'olist_orders_dataset.csv', 'olist_products_dataset.csv', \
         'olist_sellers_dataset.csv', 'product_category_name_translation.csv']
new_names = ['customer_data', 'geolocation_data', 'order_items_data', 'order_payments_data', 'order_reviews_data',\
             'orders_data', 'products_data', 'sellers_data', 'product_translation']

## Bulk Load CSV Files into SQL Tables

- Define and execute a function to iterate over all CSV files and load them into the PostgreSQL database.
- Each CSV is read as a DataFrame and written to a corresponding SQL table using `pandas.to_sql()`.
- Existing tables with the same name are replaced to ensure consistency.


In [4]:
def read_archives(file_list, table_names):
    for file_list, table_names in zip(file_list, table_names):
        df = pd.read_csv(file_list)
        df.to_sql(table_names, con=engine, if_exists='replace', index=False)
read_archives(files, new_names)

## Preview Loaded Tables from SQL Database

- Run SQL `SELECT *` queries to preview the first few rows of each loaded table.
- Confirm that all datasets were successfully imported and structured correctly.
- Use `.head()` to display sample records and validate schema integrity.


In [5]:
pd.read_sql("""SELECT * FROM customer_data""", engine).head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [6]:
pd.read_sql("""SELECT * FROM geolocation_data""", engine).head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [7]:
pd.read_sql("""SELECT * FROM order_items_data""", engine).head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [8]:
pd.read_sql("""SELECT * FROM order_payments_data""", engine).head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [9]:
pd.read_sql("""SELECT * FROM orders_data""", engine).head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [10]:
pd.read_sql("""SELECT * FROM products_data""", engine).head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [11]:
pd.read_sql("""SELECT * FROM order_reviews_data""", engine).head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [12]:
pd.read_sql("""SELECT * FROM sellers_data""", engine).head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [13]:
pd.read_sql("""SELECT * FROM product_translation""", engine).head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


## Translate Product Categories to English

- Modify the `products_data` table by removing (if exists) and adding a new column `product_category_en`.
- Initially populate `product_category_en` with values from the original Portuguese column.
- Use a SQL `UPDATE ... FROM` join to map and overwrite the translated English category names from the `product_translation` table.
- This ensures category names are standardized and ready for analysis in English.


In [14]:
query = """
ALTER TABLE products_data DROP COLUMN IF EXISTS product_category_en;
ALTER TABLE products_data ADD COLUMN product_category_en TEXT;

UPDATE products_data
SET product_category_en = product_category_name;

UPDATE products_data A
SET product_category_en = B.product_category_name_english
FROM product_translation B
WHERE A.product_category_name = B.product_category_name;
"""

with engine.begin() as conn:
    conn.execute(text(query))

## Normalize Brazilian State Names

- Create a mapping dictionary to convert state abbreviations (e.g., 'SP') to full names (e.g., 'Sao Paulo').
- Iterate over the mapping and update the `seller_state` and `customer_state` fields in their respective tables.
- This improves readability and consistency in regional analysis and visualizations.


In [15]:
state_map = {
    'SP': 'Sao Paulo', 'RJ': 'Rio de Janeiro', 'MG': 'Minas Gerais', 'RS': 'Rio Grande do Sul',
    'PR': 'Parana', 'SC': 'Santa Catarina', 'BA': 'Bahia', 'DF': 'Distrito Federal',
    'GO': 'Goias', 'ES': 'Espirito Santo', 'PE': 'Pernambuco', 'CE': 'Ceara',
    'PA': 'Para', 'MT': 'Mato Grosso', 'MA': 'Maranhao', 'MS': 'Mato Grosso do Sul',
    'PB': 'Paraiba', 'PI': 'Piaui', 'RN': 'Rio Grande do Norte', 'AL': 'Alagoas',
    'SE': 'Sergipe', 'TO': 'Tocantins', 'RO': 'Rondonia', 'AM': 'Amazonas',
    'AC': 'Acre', 'AP': 'Amapa', 'RR': 'Roraima'
}

with engine.begin() as conn:
    for abbr, full in state_map.items():
        conn.execute(text("""
            UPDATE sellers_data
            SET seller_state = :full
            WHERE seller_state = :abbr;
        """), {'abbr': abbr, 'full': full})

        conn.execute(text("""
            UPDATE customer_data
            SET customer_state = :full
            WHERE customer_state = :abbr;
        """), {'abbr': abbr, 'full': full})

## Extract Order Dates for Temporal Analysis

- Select the `order_purchase_timestamp` column from the `orders_data` table.
- Cast the timestamp to `DATE` format to simplify time-based analysis (e.g., daily, monthly trends).
- Store the result in a DataFrame for further use in temporal aggregations and visualizations.
- This "calendar" table will be used in Power BI to create relationships with other tables, enabling more organized and flexible time-based visualizations.


In [16]:
query = """
SELECT order_purchase_timestamp::DATE AS "Order Date"
FROM orders_data
"""
OrderDates = pd.read_sql(query, engine)

## Aggregate Daily Order Counts

- Query the `orders_data` table to count the number of orders per purchase date.
- Group results by the date extracted from `order_purchase_timestamp`.
- Store the aggregated data in a DataFrame for trend analysis.
- This summary will support Power BI visualizations showing order volume over time and help identify peak sales periods.

In [17]:
query = """
SELECT order_purchase_timestamp::DATE, count(order_id)  FROM orders_data
GROUP BY 1
"""
OrdersCount = pd.read_sql(query, engine)

## Analyze Product Reviews by Category and Date

- Join product, order item, order review, and order details tables to combine sales and review data.
- Select product categories (in English), review scores, and order purchase dates.
- Categorize reviews into "Good" (score ≥ 4) and "Bad" (score ≤ 3) for simplified sentiment analysis.
- This dataset supports Power BI visualizations that correlate product categories with customer satisfaction over time.


In [18]:
query = """
SELECT product_category_en, review_score, 
order_purchase_timestamp::DATE,
	CASE 
		WHEN review_score >= 4 Then 'Good'
		WHEN review_score <= 3 THEN 'Bad'
	END Review
	
FROM products_data pd JOIN order_items_data  oi
ON pd.product_id = oi.product_id
JOIN order_reviews_data ore 
ON oi.order_id = ore.order_id 
JOIN orders_data od
ON oi.order_id = od.order_id
"""
Reviews = pd.read_sql(query, engine)

## Extract Revenue by Product Category and Order Date

- Join product, order items, and orders tables to associate sales revenue with product categories and purchase dates.
- Select the English product category, rounded item price as revenue, and order purchase date.
- Order results by date to facilitate time series analysis.
- This data will be used in Power BI to analyze sales trends by category over time.


In [19]:
query = """
SELECT pd.product_category_en, 
round(oi.price::numeric, 2) AS "Revenue",
od.order_purchase_timestamp::DATE AS "Order Date"
FROM products_data pd
JOIN order_items_data oi
ON pd.product_id = oi.product_id
JOIN orders_data od
ON oi.order_id = od.order_id
ORDER BY 3;
"""
CatSales_payments = pd.read_sql(query, engine)

## Extract Revenue and Freight Costs by Product Category and Order Date

- Join product, order items, and orders tables to gather sales and shipping cost data.
- Select English product category, rounded revenue, rounded freight costs, and formatted order purchase date.
- Order results by date to support chronological analysis.
- This dataset enables Power BI visualizations comparing product revenues and associated freight costs over time.


In [20]:
query = """
SELECT product_category_en, ROUND(price::numeric, 2) AS "Revenue",
ROUND((freight_value::numeric), 2) AS "Freight Costs",
TO_CHAR(order_purchase_timestamp::timestamp, 'YYYY-MM-DD') AS "Order_Date"
FROM products_data pd JOIN order_items_data oi
ON pd.product_id=oi.product_id
JOIN orders_data od
ON oi.order_id = od.order_id
ORDER BY 4;
"""

MeanCatSales_payments = pd.read_sql(query, engine)

## Aggregate Total Orders by Date

- Group orders by purchase date formatted as 'YYYY-MM-DD'.
- Count the total number of orders per day.
- Order results by total orders to identify dates with highest and lowest sales volume.
- This aggregation supports Power BI analysis to detect peak sales days and trends over time.

In [21]:
query = """
SELECT TO_CHAR(order_purchase_timestamp::timestamp, 'YYYY-MM-DD') AS "Order year", 
count(order_purchase_timestamp) AS "Total Orders"
FROM orders_data
GROUP BY "Order year"
ORDER BY "Total Orders"
"""

Most_purchased_dates = pd.read_sql(query, engine)

-Top estados o ciudades con mayor volumen de ventas.

## Analyze Sales Performance by Customer State and Date

- Join customer, orders, and order items tables to link sales data with customer locations.
- Aggregate the number of distinct orders and total revenue by customer state and order date.
- Group results by state and date to capture regional sales trends over time.
- This data supports Power BI visualizations highlighting top-performing regions and temporal sales patterns.


In [22]:
query = """
SELECT 
  cd.customer_state AS "Customers State",
  COUNT(DISTINCT od.order_id) AS "Orders Quantity",
  SUM(oi.price::numeric) AS Revenue,
  DATE(order_purchase_timestamp) AS date
FROM customer_data cd 
JOIN orders_data od ON cd.customer_id = od.customer_id
JOIN order_items_data oi ON od.order_id = oi.order_id
GROUP BY cd.customer_state, DATE(order_purchase_timestamp)
"""
TopCitySales = pd.read_sql(query, engine)

## Calculate Average Delivery Time by State for Customers and Sellers

- Use Common Table Expressions (CTEs) to calculate:
  - Average delivery time and total orders **to customers** by state.
  - Average delivery time and total orders **from sellers** by state.
- Join both customer and seller delivery statistics on state, using a full outer join to include all states.
- This analysis helps identify regional differences in delivery performance for both buyers and sellers.
- The results can be visualized in Power BI to compare delivery efficiency across states.


In [23]:
query = """
WITH customer_avg AS (
  SELECT
    cd.customer_state AS state,
    COUNT(*) AS total_orders_to_state,
    ROUND(AVG(EXTRACT(EPOCH FROM od.order_delivered_customer_date::timestamp 
                      - od.order_purchase_timestamp::timestamp) / 86400.0), 1) AS avg_delivery_customer_state
  FROM orders_data od
  JOIN customer_data cd ON od.customer_id = cd.customer_id
  WHERE od.order_delivered_customer_date IS NOT NULL 
    AND od.order_purchase_timestamp IS NOT NULL
  GROUP BY cd.customer_state
),

seller_avg AS (
  SELECT
    sd.seller_state AS state,
    COUNT(*) AS total_orders_from_state,
    ROUND(AVG(EXTRACT(EPOCH FROM od.order_delivered_customer_date::timestamp 
                      - od.order_purchase_timestamp::timestamp) / 86400.0), 1) AS avg_delivery_seller_state
  FROM orders_data od
  JOIN order_items_data oi ON od.order_id = oi.order_id
  JOIN sellers_data sd ON sd.seller_id = oi.seller_id
  WHERE od.order_delivered_customer_date IS NOT NULL 
    AND od.order_purchase_timestamp IS NOT NULL
  GROUP BY sd.seller_state
)

SELECT
  COALESCE(c.state, s.state) AS state,
  c.total_orders_to_state,
  c.avg_delivery_customer_state,
  s.total_orders_from_state,
  s.avg_delivery_seller_state
FROM customer_avg c
FULL OUTER JOIN seller_avg s ON c.state = s.state
ORDER BY state;
"""

AvgArrivalTime = pd.read_sql(query, engine)

## Customer Purchase Frequency Distribution

- Calculate the number of customers grouped by their total number of purchases.
- Compute the percentage each group represents out of the total distinct customers.
- This distribution highlights buying behavior patterns and customer loyalty levels.
- Results can be visualized in Power BI to identify segments with high or low purchase frequencies.


In [25]:
query = """
SELECT count(customer_unique_id) AS "Number of Customers", 
"Orders_Count" AS "Number of customer purchases",
ROUND((100.0 / 
(
SELECT count(DISTINCT C.customer_unique_id)
FROM customer_data C 
JOIN orders_data O ON C.customer_id = O.customer_id)
) * count(*), 3) AS "Percentage of purchases"

FROM (
SELECT C.customer_unique_id, count(O.order_id) AS "Orders_Count"
FROM customer_data C JOIN orders_data O
ON C.customer_id = O.customer_id
GROUP BY C.customer_unique_id
)

GROUP BY "Number of customer purchases"
ORDER BY 1 DESC;
"""
PercentagePurchases = pd.read_sql(query, engine)

## Analyze Payment Types and Average Payment Amounts

- Group payments by `payment_type` to calculate:
  - Average payment amount per type.
  - Total count of purchases for each payment type.
  - Percentage share of each payment type relative to all payments.
- Order results by average payment amount in descending order.
- This analysis provides insights into customer payment preferences and transaction sizes, useful for financial reporting in Power BI.


In [26]:
query = """
SELECT payment_type, ROUND(AVG(payment_value)::numeric, 2) AS "Avg Payment Amount",
count(payment_type) AS "Purchase Count",
ROUND((100.0 / (SELECT count(*) FROM order_payments_data))
* (count(payment_type)), 2) AS "Payment_type_percentage"
FROM order_payments_data
GROUP BY payment_type
ORDER BY 2 DESC;
"""
PayemntTypes = pd.read_sql(query, engine)

## Customer Cohort Analysis: Monthly Retention and Revenue

- Identify each customer's first purchase month to define cohorts.
- Assign all subsequent orders to corresponding cohorts based on customer and order months.
- Aggregate monthly active customers and total revenue by cohort and order month.
- Calculate months elapsed since the cohort’s first purchase to track retention over time.
- This analysis enables Power BI visualizations of customer retention, revenue growth, and cohort behavior patterns.

In [27]:
query = """
WITH first_orders AS (
  SELECT
    c.customer_unique_id,
    MIN(o.order_purchase_timestamp) AS first_order_date
  FROM orders_data o
  JOIN customer_data c
    ON o.customer_id = c.customer_id
  GROUP BY c.customer_unique_id
),
cohorts AS (
  SELECT
    customer_unique_id,
    DATE_TRUNC('month', first_order_date::timestamp) AS cohort_month
  FROM first_orders
),
orders_with_cohort AS (
  SELECT
    o.order_id,
    c.customer_unique_id,
    DATE_TRUNC('month', o.order_purchase_timestamp::timestamp) AS order_month,
    cohorts.cohort_month
  FROM orders_data o
  JOIN customer_data c
    ON o.customer_id = c.customer_id
  JOIN cohorts
    ON c.customer_unique_id = cohorts.customer_unique_id
),
payments_with_cohort AS (
  SELECT
    oc.cohort_month,
    oc.order_month,
    COUNT(DISTINCT oc.customer_unique_id) AS customers_active,
    ROUND(SUM(p.payment_value)::numeric, 2) AS revenue
  FROM orders_with_cohort oc
  JOIN order_payments_data p
    ON oc.order_id = p.order_id
  GROUP BY oc.cohort_month, oc.order_month
)
SELECT
  cohort_month::DATE,
  order_month::DATE,
  DATE_PART('year', order_month) * 12 + DATE_PART('month', order_month)
    - (DATE_PART('year', cohort_month) * 12 + DATE_PART('month', cohort_month)) AS months_since_cohort,
  customers_active,
  revenue
FROM payments_with_cohort
ORDER BY cohort_month, order_month;
"""

Cohort = pd.read_sql(query, engine)

## Cohort Retention Analysis Over Time

- Build on customer cohorts defined by their first purchase month.
- Calculate the number of active customers and revenue for each month since their cohort started.
- Compute the percentage of total active customers retained at each month interval.
- This aggregated retention metric helps visualize customer loyalty and lifetime value trends in Power BI.


In [28]:
query = """
WITH first_orders AS (
  SELECT
    c.customer_unique_id,
    MIN(o.order_purchase_timestamp) AS first_order_date
  FROM orders_data o
  JOIN customer_data c ON o.customer_id = c.customer_id
  GROUP BY c.customer_unique_id
),

cohorts AS (
  SELECT
    customer_unique_id,
    DATE_TRUNC('month', first_order_date::timestamp) AS cohort_month
  FROM first_orders
),

orders_with_cohort AS (
  SELECT
    o.order_id,
    c.customer_unique_id,
    DATE_TRUNC('month', o.order_purchase_timestamp::timestamp) AS order_month,
    cohorts.cohort_month
  FROM orders_data o
  JOIN customer_data c ON o.customer_id = c.customer_id
  JOIN cohorts ON c.customer_unique_id = cohorts.customer_unique_id
),

payments_with_cohort AS (
  SELECT
    oc.cohort_month,
    oc.order_month,
    COUNT(DISTINCT oc.customer_unique_id) AS customers_active,
    ROUND(SUM(p.payment_value)::numeric, 2) AS revenue
  FROM orders_with_cohort oc
  JOIN order_payments_data p ON oc.order_id = p.order_id
  GROUP BY oc.cohort_month, oc.order_month
),

base_data AS (
  SELECT
    cohort_month::DATE,
    order_month::DATE,
    DATE_PART('year', order_month) * 12 + DATE_PART('month', order_month)
      - (DATE_PART('year', cohort_month) * 12 + DATE_PART('month', cohort_month)) AS months_since_cohort,
    customers_active,
    revenue
  FROM payments_with_cohort
),

total_customers AS (
  SELECT SUM(customers_active) AS total FROM base_data
)

SELECT
  months_since_cohort,
  SUM(customers_active) AS total_active,
  ROUND(SUM(customers_active) / total_customers.total * 100.0, 2) AS percent_of_total
FROM base_data, total_customers
GROUP BY months_since_cohort, total_customers.total
ORDER BY months_since_cohort;
"""
CohortRet = pd.read_sql(query, engine)

## Analyze Order Status by Customer City and Date

- Join orders and customer data to associate order statuses with customer locations.
- Group by city, order status, and purchase date to count orders per status in each city daily.
- This dataset supports Power BI visualizations tracking order fulfillment and cancellation trends regionally over time.

In [29]:
query = """
SELECT od.order_status, count(od.order_status), cd.customer_city,
DATE(order_purchase_timestamp)
FROM orders_data od JOIN customer_data cd
ON od.customer_id = cd.customer_id
GROUP BY cd.customer_city, od.order_status, DATE(order_purchase_timestamp)
"""
Status_by_city = pd.read_sql(query, engine)

## Analyze Order Status by Product Category and Date

- Join orders, order items, and products tables to link order statuses with product categories.
- Group by product category, order status, and purchase date to count orders per status daily.
- This data facilitates Power BI analysis of fulfillment performance and issues across different product categories over time

In [30]:
query = """
SELECT od.order_status, count(od.order_status) AS "Order_status",
pd.product_category_en AS "Product_Category" , DATE(od.order_purchase_timestamp)
FROM orders_data od JOIN order_items_data oi
ON od.order_id = oi.order_id
JOIN products_data pd 
ON oi.product_id = pd.product_id
GROUP BY pd.product_category_en, od.order_status, DATE(od.order_purchase_timestamp)
"""

Status_by_cat = pd.read_sql(query, engine)

## Export All DataFrames to CSV Files

- Define a function that scans global variables for pandas DataFrames.
- Export each detected DataFrame to a CSV file in the specified directory.
- Skip internal or output variables based on naming patterns.
- Automate saving intermediate results for use in Power BI or further analysis.

In [31]:
def export_all_dataframes(path=''):

    if path and not os.path.exists(path):
        os.makedirs(path)

    frame = inspect.currentframe()
    global_vars = frame.f_back.f_globals

    for var_name, var_value in global_vars.items():
        if isinstance(var_value, pd.DataFrame):
            if re.match(r'^(_|Out)', var_name):
                continue
            filename = f"{path}{var_name}.csv"
            var_value.to_csv(filename, index=False)
            print(f'Guardado: {filename}')

export_all_dataframes(path='Dataframes/') 

Guardado: Dataframes/OrderDates.csv
Guardado: Dataframes/OrdersCount.csv
Guardado: Dataframes/Reviews.csv
Guardado: Dataframes/CatSales_payments.csv
Guardado: Dataframes/MeanCatSales_payments.csv
Guardado: Dataframes/Most_purchased_dates.csv
Guardado: Dataframes/TopCitySales.csv
Guardado: Dataframes/AvgArrivalTime.csv
Guardado: Dataframes/MaxPurchases.csv
Guardado: Dataframes/PercentagePurchases.csv
Guardado: Dataframes/PayemntTypes.csv
Guardado: Dataframes/Cohort.csv
Guardado: Dataframes/CohortRet.csv
Guardado: Dataframes/Status_by_city.csv
Guardado: Dataframes/Status_by_cat.csv


## Conclusions and Strategic Insights

The following conclusions are based on the data exploration, SQL queries, and visual analysis through the accompanying Power BI dashboard:


1. **Robust Growth Trajectory**  
   - Over the 24-month span from September 2016 to August 2018, Olist processed **99,440 orders** and generated **\$1.26 million** in revenue. Revenue jumped from 49.8K in late-2016 to 6.17M in 2017 and \$7.39 M in 2018, indicating strong market adoption and effective scaling of operations.  
   - **Insight:** Sustained year-over-year growth suggests that marketing efforts, marketplace partnerships, and customer acquisition strategies were successful. Future focus on customer retention (see point 6) could amplify lifetime value and drive even higher revenue per cohort.

2. **Seasonality and Promotional Impact**  
   - Distinct spikes in **May** (Health & Beauty) and **November** (Watches & Gifts, peaking November 24) closely align with Brazilian holiday and promotional calendars Mother’s Day in May and Black Friday in November.  
   - **Theory:** Coordinated marketing campaigns or flash sales during these months likely boosted order volume. Investing in targeted promotions aligned with local festivities could further amplify returns.

3. **Dominant Categories—Revenue vs. Volume**  
   - **Health & Beauty** leads total revenue across all years (≈\$1.26 M), a sign of high average order value or strong repeat purchases. However, **Bed & Bath Table** tops order volume (11,115 orders; 15.5 %), suggesting it’s a low-ticket, high-frequency category.  
   - **Strategic Insight:** Bundling low-ticket home goods with higher-margin health & beauty products or cross-selling strategies could increase basket size and margin.

4. **Customer Satisfaction Patterns**  
   - Of **112,370** total reviews, **75%** were positive (≥4 stars) and **25%** negative (≤3 stars). The **Bed & Bath Table** segment not only had the most orders but also the highest counts of both positive (7,916) and negative (3,221) reviews reflecting mixed product quality perceptions amid high volume.  
   - **Theory:** Large-volume categories often face quality consistency challenges. Implementing stricter vendor onboarding or post-sale surveys could pinpoint and mitigate negative experiences.

5. **Regional Market Concentration**  
   - **São Paulo** accounts for **41 k orders** and \$5.20 M in revenue (≈41 % of total), with **Rio de Janeiro** (13 k orders; \$1.82 M) and **Minas Gerais** (12 k orders; \$1.59 M) rounding out the top three. Conversely, remote states like Roraima had as few as 7,829 in sales.  
   - **Insight:** High urban density markets dominate, while frontier regions represent untapped potential. Tailoring logistics and localized promotions to under-penetrated states could unlock new growth.

6. **Delivery Performance Gaps**  
   - Average delivery time to customers is **18.8 days**, while sellers average **15.2 days** in dispatch. São Paulo leads with rapid fulfillment (8.8 days customer / 12.3 days seller), but states like Bahia show a 5 day discrepancy (19.3 vs. 13.8), indicating last-mile challenges.  
   - **Recommendation:** Partnering with regional carriers or establishing distribution hubs in slower states could reduce delivery times, improve customer satisfaction, and lower negative review rates.

7. **Payment Method Preferences**  
   - **Credit card** dominates at **73.9 %**, followed by Boleto (19.0 %), Voucher (5.6 %), and Debit card (1.5 %). Boleto’s high share underscores Brazil’s unique payment landscape where many consumers still favor cash-based or deferred payments.  
   - **Opportunity:** Introducing installment plans or “Buy Now, Pay Later” options could boost average order values among boleto users and capture additional market share.

8. **Customer Retention Challenges**  
   - **96.9 %** of customers made only one purchase; only **2.9 %** returned for a second, and just **0.2 %** for a third. Cohort analysis shows a drop from **98.0 %** first-purchase cohort to **0.5 %** in month 2 and below **0.3 %** by month 3.  
   - **Insight:** While acquisition is strong, repeat purchase rates are low. A lifecycle marketing program automated email reminders, loyalty rewards, or personalized offers could nudge customers toward a second purchase and reduce churn.

9. **Order Status Dynamics**  
   - Highest cancellation counts occur in São Paulo (140), Rio de Janeiro (48), and Belo Horizonte (17), mirroring the highest order volumes.  
   - **Interpretation:** Cancellation rates are proportional to order volume, but zeroing in on cancellation reasons (e.g., stockouts, payment failures) could reveal process improvements to reduce this waste.

10. **Holistic Strategic Roadmap**  
    - **Optimize Core Regions:** Double down on top-performing states with tailored logistics and marketing investments.  
    - **Enhance Retention:** Launch loyalty programs and targeted re-engagement campaigns to boost repeat purchase rates.  
    - **Improve Last-Mile Logistics:** Address delivery gaps in slower states to reduce negative reviews and cancellations.  
    - **Expand Payment Options:** Innovate payment solutions to capture boleto and voucher segments.  
    - **Quality Control in High-Volume Categories:** Strengthen vendor management in Bed & Bath Table to maintain product consistency and minimize negative reviews.
 